In [5]:
# Import libraries 
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os
#import urllib
import re
import shutil
import urllib.request as request
from contextlib import closing
from glob import glob

In [ ]:
import urllib.request

In [135]:
local_IAP_dir = './IAPs/norcal/incidents/'

In [136]:
ensure_dir_exists('./IAPs/norcal/incidents/')

## Helper functions

In [24]:
def url_list_from_html(html):
    '''
    Use a regex pattern to find only the URLs (links) in some html.
    '''
    return re.findall(r"""<\s*a\s*href=["']([^=]+)["']""", html.decode("utf-8"))

def ensure_dir_exists(path):
    '''
    Create directory structure given by path if it doesn't already exist
    '''
    if not os.path.exists(path):
        os.makedirs(path)
        
def get_local_IAPs(local_IAP_dir,suffix_pattern='*.pdf'):
    available_IAPs = []
    for incident in glob(os.path.join(local_IAP_dir,'*')):
        for incident_date in glob(os.path.join(incident,'*')):
            available_IAPs.extend(glob(os.path.join(incident_date,suffix_pattern)))
    return available_IAPs

Download Params

In [165]:
region_agency = 'calif_n/!CALFIRE/'
base_url = 'https://ftp.nifc.gov/public/incident_specific_data/{}'.format(region_agency)
State_Code = 'CA'
download_corrected_IAPs = False

In [223]:
with urllib.request.urlopen(base_url) as response:
    urllist_year = url_list_from_html(response.read())
    
year_urls = [s for s in urllist if "_Incidents" in s]
    
for year_url in year_urls:
    with urllib.request.urlopen(base_url + year_url) as response:
        urllist_incidents = url_list_from_html(response.read())
        
    # Get list of incidents using state code (first two chars in incident code) and exclude templates using 'xxx' pattern
    incident_urls = [s for s in urllist_incidents if State_Code == s[:2] and s[3:6] != 'xxx']
    for incident_url in incident_urls:
        try:
            #Get list of links available inside the IAP directory for this incident (if the IAP dir exists)
            try:
                base_year_indident_IAP_url = base_url + year_url + incident_url + 'IAP/'
                with urllib.request.urlopen(base_year_indident_IAP_url) as response:
                    available_IAP_dirs = url_list_from_html(response.read())
            except:
                base_year_indident_IAP_url = base_url + year_url + incident_url + 'IAPs/'
                with urllib.request.urlopen(base_year_indident_IAP_url) as response:
                    available_IAP_dirs = url_list_from_html(response.read())
                    
            #Just take the links that contain dates (i.e. the IAPs for each available Operational Period)
            #### Could probably use a better way to validate the date
            IAP_date_urls = [s for s in available_IAP_dirs if s[2] == '-' and s[5] == '-']

            #Go through each of these directory links
            for IAP_date_url in IAP_date_urls:
                try:
                    with urllib.request.urlopen(base_year_indident_IAP_url + IAP_date_url) as response:
                        #Get a list of all links available in this directory that contain the '.pdf' suffix
                        pdf_list = [s for s in url_list_from_html(response.read()) if s[-4:] == '.pdf']
                        
                        #PDFs appear available, create the corresponding local dir
                        download_dir = os.path.join(local_IAP_dir,incident_url,IAP_date_url)
                        ensure_dir_exists(download_dir)

                        # Check if the uncorrected or corrected IAPs should be downloaded. 
                        if download_corrected_IAPs:
                            valid_pdf_list = [s for s in pdf_list if 'CORRECTED' in s]
                        else:
                            valid_pdf_list = [s for s in pdf_list if 'CORRECTED' not in s]

                        #Check length of pdfs to download
                        if len(valid_pdf_list) == 0:
                            print('No IAP is present for {} on {}. Empty directory?'.format(incident_url,IAP_date_url))
                        elif len(valid_pdf_list) > 1:
                            print('More than one IAP is present for {} on {}. Only the first in the list will be downloaded.'.format(incident_url,IAP_date_url))

                        #Try downloading the given IAP
                        download_url = base_year_indident_IAP_url + IAP_date_url + valid_pdf_list[0]
                        download_local_path = os.path.join(download_dir,valid_pdf_list[0])
                        #download_url = base_url + year_url + incident_url + IAP_date_url + pdf_list[0]
                        #print(download_url)
                        if not os.path.isfile(download_local_path):
                            try:
                                with closing(request.urlopen(download_url)) as r:
                                    with open(download_local_path, 'wb') as f:
                                        shutil.copyfileobj(r, f)
                            except Exception as e:
                                print('Download failed for {} with execption:'.format(download_url))
                                print(e)
                            print('Downloaded {}'.format(download_local_path))
                        else:
                            print('File {} already exists'.format(download_local_path))

                except Exception as e:
                    print('Could not access url {} with execption:'.format(bbase_year_indident_IAP_url + IAP_date_url))
                    print(e)
        except Exception as e:
            print('IAP directory does not appear to exist for {}'.format(base_url + year_url + incident_url))
            print(e)
    

IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-BTU-005638-Cedar/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-LNU-002934-Yellow/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-LNU-010188-McCabe/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-SCU-005452-Morgan/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-SHU-008265-Clover/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2013_Incidents/CA-TG

IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2017_Incidents/CA-AEU-021652_Latrobe/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2017_Incidents/CA-BTU-010975_Wall/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2017_Incidents/CA-BTU-013737_Ponderosa/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2017_Incidents/CA-BTU-015933_Cherokee/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2017_Incidents/CA-CDF-000114_Spillway/
HTTP Error 404: Not Found
IAP directory does not appear to exist for https://ftp.nifc.gov/public/incident_specific_data/calif_n/!CALFIRE/2017_Incident

In [177]:
if not os.path.isfile(os.path.join(download_dir,valid_pdf_list[0],'test')):
    print('yes')

yes


## Convert downloaded IAP PDFs to text

In [224]:
available_IAPs = get_local_IAPs(local_IAP_dir)

In [225]:
available_IAPs

['./IAPs/norcal/incidents/CA-LNU-009502_County/07-04-2018/COUNTY_IAP_07-04-18.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-08-2018/COUNTY_IAP_07-08-2018.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-09-2018/COUNTY_IAP_07-09-18%20Run.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-05-2018/COUNTY_Corrected_7-5-18.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-02-2018/COUNTY_IAP_07-02-2018_reduced.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-03-2018/20180703_County_IAP_reduced.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-12-2018_3-DAY/COUNTY_3-DAY_07-12-18_3_Optimized.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-01-2018/COUNTY_IAP_07-01-2018.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-10-2018/COUNTY_IAP_07-10-18%20Run.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-06-2018/COUNTY_IAP_07-06-18.pdf',
 './IAPs/norcal/incidents/CA-LNU-009502_County/07-07-2018/COUNTY_IAP_07-07-18.pdf',
 './IAPs/norcal/in

In [ ]:
for IAP_path in available_IAPs:
    iap2txt(IAP_path)

Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-04-2018/COUNTY_IAP_07-04-18.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-08-2018/COUNTY_IAP_07-08-2018.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-09-2018/COUNTY_IAP_07-09-18%20Run.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-05-2018/COUNTY_Corrected_7-5-18.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-02-2018/COUNTY_IAP_07-02-2018_reduced.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-03-2018/20180703_County_IAP_reduced.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-12-2018_3-DAY/COUNTY_3-DAY_07-12-18_3_Optimized.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-01-2018/COUNTY_IAP_07-01-2018.txt
Completed conversion of ./IAPs/norcal/incidents/CA-LNU-009502_County/07-10-2018/COUNTY_IAP_07-10-18%20Run.txt


In [215]:
def iap2txt(PDF_file):
    saved_image_paths, outpath = pdf2jpeg(PDF_file)
    jpeg2text(saved_image_paths, outpath)
    
def pdf2jpeg(PDF_file,overwrite_images=False):
    # Path of the pdf 
    #PDF_file = "./IAPs/incidents/IAP_KINCADE_10252019.pdf"
    basename = os.path.basename(PDF_file).split('.')
    iap_name = basename[0]
    outpath = os.path.join(os.path.dirname(PDF_file),iap_name+".txt")

    imagefile_dir = PDF_file[:-4]+'_page_images'
    ensure_dir_exists(imagefile_dir)
    

    ''' 
    Part #1 : Converting PDF to images 
    '''

    
    # Store all the pages of the PDF in a variable 
    pages = convert_from_path(PDF_file, 500) 

    # Counter to store images of each page of PDF to image 
    image_counter = 1
    saved_image_paths = []

    # Iterate through all the pages stored above 
    for page in pages: 

        # Declaring filename for each page of PDF as JPG 
        # For each page, filename will be: 
        # PDF page 1 -> page_1.jpg 
        # PDF page 2 -> page_2.jpg 
        # PDF page 3 -> page_3.jpg 
        # .... 
        # PDF page n -> page_n.jpg 
        #filename = iap_name+"_page_"+str(image_counter)+".jpg"
        filepath = os.path.join(imagefile_dir,iap_name+"_page_"+str(image_counter)+".jpg")
        
        # Save the image of the page in system 
        saved_image_paths.append(filepath)
        
        if not os.path.isfile(filepath):
            page.save(filepath, 'JPEG') 
        elif overwrite_images:
            page.save(filepath, 'JPEG')

        # Increment the counter to update filename 
        image_counter = image_counter + 1
        
    return saved_image_paths, outpath

def jpeg2text(saved_image_paths,outpath):

    ''' 
    Part #2 - Recognizing text from the images using OCR 
    '''
    # Variable to get count of total number of pages 
    #filelimit = image_counter-1

    # Creating a text file to write the output 
    #outfile = iap_name+".txt"
    
    # Open the file in append mode so that 
    # All contents of all images are added to the same file 
    #f = open(outfile, "a") 
    with open(outpath,"a") as file:

        # Iterate from 1 to total number of pages
        for image_path in saved_image_paths:
        #for i in range(1, filelimit + 1): 

            # Set filename to recognize text from 
            # Again, these files will be: 
            # page_1.jpg 
            # page_2.jpg 
            # .... 
            # page_n.jpg 
            #filename = "page_"+str(i)+".jpg"

            # Recognize the text as string in image using pytesserct 
            text = str(((pytesseract.image_to_string(Image.open(image_path))))) 

            # The recognized text is stored in variable text 
            # Any string processing may be applied on text 
            # Here, basic formatting has been done: 
            # In many PDFs, at line ending, if a word can't 
            # be written fully, a 'hyphen' is added. 
            # The rest of the word is written in the next line 
            # Eg: This is a sample text this word here GeeksF- 
            # orGeeks is half on first line, remaining on next. 
            # To remove this, we replace every '-\n' to ''. 
            text = text.replace('-\n', '')	 

            # Finally, write the processed text to the file. 
            file.write(text) 

    # Close the file after writing all the text. 
    #f.close() 
    print('Completed conversion of {}'.format(outpath))
